In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from ast import literal_eval
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "9"

In [ ]:
num_workers=1
batch_size=100
lr=0.01
num_epoch=200

In [2]:
num_level=1
depth_batch=1
hide_neuron=32

annotation_train = 'data/data_se/data_train.txt'
annotation_val = 'data/data_se/data_test.txt'
var_size=1370+1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class MyDataset(Dataset):
    def __init__(self, annotation):
        self.landmarks_frame = pd.read_csv(annotation)
    def __len__(self):
        return len(self.landmarks_frame)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        i_data = literal_eval(self.landmarks_frame.iloc[idx, 1])

        return i_data
train_dataset = MyDataset(annotation_train)
val_dataset = MyDataset(annotation_val)

In [4]:
def pad_length(b):
    # build vectorized sequence
    v = b
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=var_size-1) for t in v]),
    )

In [5]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, drop_last=True, shuffle=True,collate_fn=pad_length)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=True, drop_last=True, shuffle=False,collate_fn=pad_length)

In [6]:
from lstm import LSTM

In [7]:
net=LSTM(var_size,hide_neuron,num_level)
net=net.to(device)
Loss = nn.CrossEntropyLoss(ignore_index=var_size-1)
optimizer = optim.AdamW(net.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.5,patience=5)

In [8]:
def micro_batch(input,index=0,input_size=1,var_size=1):
    sting=input[:,index:index+input_size]#.type(torch.int32)
    label_ind=input[:,index+1:index+input_size+1]#.type(torch.int32)
    size=np.array([sting.shape[0]*sting.shape[1],var_size])
    batch=np.zeros(size)
    for i in range(sting.shape[0]):
        for j in range(sting.shape[1]):
            a=sting[i][j]
            batch[i+j][a]+=1.0
    label=torch.Tensor(label_ind).reshape(-1)
    batch=torch.Tensor(batch)
    return batch, label
    

In [52]:
best_Loss_regression=np.inf
for iter in range(num_epoch):
    for dataloader, phase in zip([train_dataloader, val_dataloader], ["T", "V"]):
        eposh_loss=0
        if phase == "T": net.train()
        if phase == "V": net.eval()
        for [data] in dataloader:
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == "T"):
                total_loss = torch.tensor([0.0], requires_grad=phase == "T").to(device)
                h0 = torch.randn(num_level,  hide_neuron).to(device)#zero
                c0 = torch.randn(num_level, hide_neuron).to(device)
                for i in range(data.shape[1]-depth_batch):
                    input_batch, target=micro_batch(data,i,depth_batch,var_size)
                    output, (h0, c0) = net(input_batch.to(device), (h0, c0))
                    target = target.long().to(device)
                    loss=Loss(output,target)
                    total_loss=total_loss+loss
                if phase == 'T':
                    # Вычислить градиенты
                    total_loss.backward()
                    # Обновить веса
                    optimizer.step()
                eposh_loss+=total_loss.item()
        if phase == "T": 
            print(f"{iter} {phase} {eposh_loss / len(dataloader):0.3f}", end=" ")
        else:
            scheduler.step(eposh_loss)
            print(f"{phase} {eposh_loss / len(dataloader):0.3f} {optimizer.param_groups[0]['lr']}")
            if eposh_loss < best_Loss_regression:
                best_model_wts = net.state_dict()
                best_model_ind = iter
                best_Loss_regression = eposh_loss
net.load_state_dict(best_model_wts)
torch.save(best_model_wts, 'model_lstm.tar')
print(best_model_ind, best_Loss_regression / len(val_dataloader))

0 T 47.996 V 39.350 0.1
1 T 39.937 V 29.732 0.1
2 T 33.223 V 26.634 0.1
3 T 29.209 V 25.313 0.1
4 T 27.049 V 24.192 0.1
5 T 25.241 V 23.571 0.1
6 T 23.420 V 22.856 0.1
7 T 23.473 V 22.927 0.1
8 T 22.449 V 22.146 0.1
9 T 22.876 V 23.324 0.1
10 T 23.146 V 23.112 0.1
11 T 22.202 V 21.547 0.1
12 T 22.049 V 21.755 0.1
13 T 22.043 V 22.683 0.1
14 T 21.424 V 22.700 0.1
15 T 21.483 V 22.111 0.1
16 T 21.829 V 22.657 0.1
17 T 21.410 V 22.498 0.05
18 T 20.355 V 21.105 0.05
19 T 19.295 V 21.126 0.05
20 T 19.363 V 21.085 0.05
21 T 19.402 V 20.985 0.05
22 T 19.430 V 21.130 0.05
23 T 19.177 V 21.389 0.05
24 T 19.344 V 21.453 0.05
25 T 19.385 V 21.039 0.05
26 T 19.062 V 21.165 0.05
27 T 19.167 V 20.791 0.05
28 T 18.894 V 20.841 0.05
29 T 18.748 V 21.596 0.05
30 T 18.897 V 20.774 0.05
31 T 19.314 V 21.667 0.05
32 T 19.535 V 21.627 0.05
33 T 18.919 V 20.785 0.05
34 T 18.806 V 21.650 0.05
35 T 18.831 V 21.264 0.05
36 T 19.275 V 21.068 0.025
37 T 18.032 V 20.710 0.025
38 T 17.628 V 20.789 0.025
39 T 17.75

In [54]:
n = 0
trye = 0
trye_3 = 0
trye_5 = 0
trye_10 = 0
trye_50 = 0
trye_500 = 0
data_test = pd.read_csv(annotation_val)
net.eval()
for d in data_test['name_id'].values:
    mas = literal_eval(d)
    h0 = torch.randn(num_level,  hide_neuron).to(device)
    c0 = torch.randn(num_level, hide_neuron).to(device)
    label=mas[-1]
    input_v=mas[:-1]
    
    for i, m in enumerate(input_v):
        buf = np.zeros((var_size),dtype = np.float32)
        buf[m] +=1
        inpyt = torch.tensor([buf]).to(device)
        with torch.set_grad_enabled(False):
            out, (h0, c0) = net(inpyt, (h0, c0))
    trye += label == out.argmax()
    trye_3 += label in out.argsort(descending=True)[0][:3]
    trye_5 += label in out.argsort(descending=True)[0][:5]
    trye_10 += label in out.argsort(descending=True)[0][:10]
    trye_50 += label in out.argsort(descending=True)[0][:50]
    trye_500 += label in out.argsort(descending=True)[0][:500]
    n+=1
print('Acc top 1', trye / n)
print('Acc top 3',trye_3 / n)
print('Acc top 5',trye_5 / n)
print('Acc top 10',trye_10 / n)
print('Acc top 50',trye_50 / n)
print('Acc top 500',trye_500 / n)

Acc top 1 tensor(0.1587, device='cuda:0')
Acc top 3 0.3353570857964531
Acc top 5 0.45087074612557915
Acc top 10 0.6137561910848378
Acc top 50 0.8780955424189167
Acc top 500 0.9871385205304362


In [55]:
n = 0
trye = 0
trye_3 = 0
trye_5 = 0
trye_10 = 0
trye_50 = 0
trye_500 = 0
data_test = pd.read_csv(annotation_val)
net.eval()
for d in data_test['name_id'].values:
    mas = literal_eval(d)
    for i, m in enumerate(mas):
        h0 = torch.randn(num_level,  hide_neuron).to(device)
        c0 = torch.randn(num_level, hide_neuron).to(device)
        label=m
        input_v=mas[:i]+mas[i+1:]
        for r in input_v:
            buf = np.zeros((var_size),dtype = np.float32)
            buf[r] +=1
            inpyt = torch.tensor([buf]).to(device)
            with torch.set_grad_enabled(False):
                out, (h0, c0) = net(inpyt, (h0, c0))
        trye += label == out.argmax()
        trye_3 += label in out.argsort(descending=True)[0][:3]
        trye_5 += label in out.argsort(descending=True)[0][:5]
        trye_10 += label in out.argsort(descending=True)[0][:10]
        trye_50 += label in out.argsort(descending=True)[0][:50]
        trye_500 += label in out.argsort(descending=True)[0][:500]
        n+=1
print('Acc top 1', trye / n)
print('Acc top 3',trye_3 / n)
print('Acc top 5',trye_5 / n)
print('Acc top 10',trye_10 / n)
print('Acc top 50',trye_50 / n)
print('Acc top 500',trye_500 / n)

Acc top 1 tensor(0.0855, device='cuda:0')
Acc top 3 0.19294074051199703
Acc top 5 0.27146959824599326
Acc top 10 0.4184294228453201
Acc top 50 0.7548096223327054
Acc top 500 0.9470092332396628
